In [1]:
import pandas as pd

# --- Configurações ---
input_file = 'Realtime_bruto.csv'
output_file = 'Realtime_clean.csv'

# Dicionário que define as colunas a serem mantidas e seus novos nomes
# Formato: { 'nome_antigo': 'nome_novo' }
columns_to_rename_and_keep = {
    'MIN': 'HORA',
    'TV GAZETA': 'GLOBO',
    'TV VITORIA': 'RECORD',
    'TV SIM': 'SBT',
    'CONTEÚDO TV/VÍDEO S/ REFERÊNCIA': 'Conteúdo de TV/Vídeo sem referência',
    'TOTAL LIGADOS': 'Total Ligados Especial'
}

try:
    # --- Passo 1: Leitura e Limpeza de Linhas ---

    # Lê o arquivo original, pulando as 6 primeiras linhas
    df = pd.read_csv(input_file, sep=';', skiprows=6)

    # Remove a última linha do DataFrame
    # Usamos .iloc[:-1] para selecionar todas as linhas, exceto a última
    df_cleaned_rows = df.iloc[:-1].copy()


    # --- Passo 2: Seleção e Renomeação de Colunas ---

    # Pega a lista das colunas originais que queremos manter
    original_columns_to_keep = list(columns_to_rename_and_keep.keys())

    # Filtra o DataFrame, mantendo apenas as colunas desejadas
    df_selected_cols = df_cleaned_rows[original_columns_to_keep]

    # Renomeia as colunas do DataFrame já filtrado
    df_final = df_selected_cols.rename(columns=columns_to_rename_and_keep)


    # --- Passo 3: Salvar o Resultado ---

    # Salva o DataFrame final, limpo e formatado, em um novo arquivo CSV
    df_final.to_csv(output_file, sep=';', index=False, encoding='utf-8')

    print("✅ Pipeline executado com sucesso!")
    print(f"Arquivo final salvo como: '{output_file}'")

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo de entrada '{input_file}' não foi encontrado.")
except KeyError as e:
    print(f"❌ ERRO: Uma ou mais colunas para renomear não foram encontradas no arquivo: {e}")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado: {e}")

✅ Pipeline executado com sucesso!
Arquivo final salvo como: 'Realtime_clean.csv'


In [2]:
import pandas as pd
import pdfplumber
import re
import os

# ==============================================================================
# PARTE 1: NOSSA FUNÇÃO PARA PROCESSAR O RUNDOWN (PDF -> DATAFRAME)
# ==============================================================================

def processar_rundown_pdf(caminho_pdf):
    """
    Função estável para extrair os dados do rundown,
    deduzindo os blocos a partir das linhas de 'Break'.
    """
    if not os.path.exists(caminho_pdf):
        print(f"ERRO: O arquivo de rundown PDF não foi encontrado em '{caminho_pdf}'")
        return None

    dados_finais = []
    bloco_numero = 1

    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            tabelas = pagina.extract_tables()
            for tabela in tabelas:
                for linha in tabela:
                    if linha and linha[0] and isinstance(linha[0], str) and 'Break' in linha[0]:
                        bloco_numero += 1
                        continue
                    
                    if len(linha) >= 10 and linha[-1] and isinstance(linha[-1], str) and linha[-1].isdigit():
                        dados_finais.append({
                            'Bloco': f"Bloco {bloco_numero:02d}",
                            'Tipo': linha[1],
                            'Retranca': linha[2],
                            'Início (Lauda)': linha[6],
                            'Fim (Lauda)': linha[7],
                            'Ordem': int(linha[9])
                        })

    if not dados_finais:
        print("Atenção: Nenhum dado de lauda válido foi extraído do PDF.")
        return None

    df = pd.DataFrame(dados_finais)
    df = df.sort_values(by='Ordem').reset_index(drop=True)
    
    print("PDF do rundown processado com sucesso!")
    return df

# ==============================================================================
# PARTE 2: SEU SCRIPT DE MERGE, AGORA INTEGRADO
# ==============================================================================

def fazer_merge_audiencia_espelho(df_espelho_orig, caminho_audiencia):
    """
    Recebe o DataFrame do espelho e o caminho do arquivo de audiência,
    e realiza a expansão e o merge em nível de segundo.
    """
    # --- Carregar o dataframe de audiência ---
    try:
        df_minamin_orig = pd.read_excel(caminho_audiencia)
        print(f"Arquivo de audiência '{caminho_audiencia}' carregado com sucesso!")
    except FileNotFoundError:
        print(f"ERRO CRÍTICO: Arquivo de audiência não encontrado - {caminho_audiencia}")
        return

    # --- 1. Expandir df_minamin para segundos ---
    print("Expandindo dados de audiência para segundos...")
    df_minamin = df_minamin_orig.copy()
    df_minamin['HORA_base'] = pd.to_datetime(df_minamin['HORA'], format='%H:%M:%S', errors='coerce').dropna()
    
    minamin_segundos_list = []
    for _, row in df_minamin.iterrows():
        minuto_base = row['HORA_base']
        for segundo_offset in range(60):
            new_row_data = row.to_dict()
            new_row_data['datetime_segundo_audiencia'] = pd.to_datetime('1900-01-01 ' + (minuto_base + pd.Timedelta(seconds=segundo_offset)).strftime('%H:%M:%S'))
            minamin_segundos_list.append(new_row_data)

    df_minamin_segundos = pd.DataFrame(minamin_segundos_list)
    
    # --- 2. Expandir df_espelho para segundos ---
    print("Expandindo dados do espelho (rundown) para segundos...")
    df_espelho = df_espelho_orig.copy()
    df_espelho.rename(columns={'Início (Lauda)': 'inicio_lauda', 'Fim (Lauda)': 'fim_lauda'}, inplace=True)
    
    df_espelho['inicio_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['inicio_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho['fim_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['fim_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho.dropna(subset=['inicio_lauda_dt', 'fim_lauda_dt'], inplace=True)
    df_espelho = df_espelho[df_espelho['fim_lauda_dt'] >= df_espelho['inicio_lauda_dt']]

    espelho_segundos_list = []
    for _, row in df_espelho.iterrows():
        for segundo_ts in pd.date_range(row['inicio_lauda_dt'], row['fim_lauda_dt'], freq='S'):
            new_row_data = row.to_dict()
            new_row_data['datetime_segundo_espelho'] = segundo_ts
            espelho_segundos_list.append(new_row_data)
    
    if not espelho_segundos_list:
        print("ERRO: Não foi possível expandir o espelho para segundos.")
        return
        
    df_espelho_segundos = pd.DataFrame(espelho_segundos_list)
    df_espelho_segundos.drop_duplicates(subset=['datetime_segundo_espelho'], keep='first', inplace=True)
    
    # --- 3. Merge em Nível de Segundo ---
    print("Realizando merge da audiência com o espelho...")
    df_merged_segundos = pd.merge(
        df_minamin_segundos,
        df_espelho_segundos,
        left_on='datetime_segundo_audiencia',
        right_on='datetime_segundo_espelho',
        how='left'
    )
    
    # --- 4. Limpeza e Salvamento ---
    colunas_para_remover = [
        'HORA_base', 'datetime_segundo_espelho', 'inicio_lauda_dt', 'fim_lauda_dt'
    ]
    df_merged_segundos.drop(columns=colunas_para_remover, inplace=True, errors='ignore')
    
    output_path = "audiencia_espelho_merged_segundos_2.0.csv"
    df_merged_segundos.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\nPROCESSO FINALIZADO! 🏆\nArquivo mergeado salvo em: {output_path}")
    
# ==============================================================================
# PONTO DE PARTIDA PRINCIPAL DO SCRIPT
# ==============================================================================

if __name__ == "__main__":
    # --- Defina os nomes dos seus arquivos de entrada aqui ---
    ARQUIVO_RUNDOWN_PDF = "rundown-display.pdf"
    ARQUIVO_AUDIENCIA = "Realtime_clean.xlsx"

    print("Iniciando pipeline de processamento e merge...")
    
    # Passo 1: Processar o PDF para obter o espelho
    df_espelho_gerado = processar_rundown_pdf(ARQUIVO_RUNDOWN_PDF)
    
    # Passo 2: Se o espelho foi gerado, prosseguir com o merge
    if df_espelho_gerado is not None:
        fazer_merge_audiencia_espelho(df_espelho_gerado, ARQUIVO_AUDIENCIA)
    else:
        print("\nO pipeline foi interrompido porque o espelho não pôde ser gerado do PDF.")

Iniciando pipeline de processamento e merge...
PDF do rundown processado com sucesso!
Arquivo de audiência 'Realtime_clean.csv' carregado com sucesso!
Expandindo dados de audiência para segundos...


KeyError: 'HORA'